In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
import numpy as np
import networkx as nx
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
# For color mapping
import matplotlib.colors as colors
import matplotlib.cm as cmx
import math
import warnings
warnings.filterwarnings("ignore")

In [2]:
DataSeed = pd.read_csv('data-seed.txt')

In [3]:
DataEdgeWeight = pd.read_csv('data-edge-weight.txt')

In [4]:
DataEdgeWeight.shape

(6688, 5)

In [5]:
edge_btw_df = pd.read_csv('explainer-vs-edge_betweenness_inspection-annotated.csv')

In [6]:
A = -0.1097 
B = 0.1064 # A for centrality, B for gnnexplainer

In [7]:
edge_btw_df

,Unnamed: 0,src,dst,id,importance,weight,weight_positive,weight_negative,edge_weight(btw),Unnamed: 9,Unnamed: 10
0,215,1,11,0,0.0,0.111897,0.108853,0.111897,0.017095,NaN,NaN
1,214,35,11,0,0.0,0.122949,0.122949,0.112641,0.017095,NaN,NaN
2,213,1,45,0,1.0,0.138937,0.124593,0.138937,0.016532,NaN,NaN
3,159,1,52,0,1.0,0.109301,0.103437,0.109301,0.016532,NaN,NaN
4,212,35,45,0,1.0,0.133006,0.133006,0.118958,0.016532,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
3339,3319,18,16,40,0.0,0.940717,0.078665,0.940717,0.000000,NaN,NaN
3340,3320,7,16,40,0.0,0.155031,0.155031,0.084675,0.000000,NaN,NaN
3341,3321,5,16,40,0.0,0.084410,0.082606,0.084410,0.000000,NaN,NaN
3342,3341,15,2,40,0.0,0.000000,0.000000,0.000000,0.000000,NaN,NaN


In [8]:
edge_btw_df['hybrid'] = B * edge_btw_df['weight'] + A * edge_btw_df['edge_weight(btw)']

In [9]:
edge_btw_df.to_csv('hybridw_edge-btww_gnnw.csv')

In [10]:
NodeLabel = pd.read_csv('annotation_publish.txt')

In [11]:
NodeLabel = NodeLabel.iloc[:, :5]

In [12]:
NodeLabel

,node_id,community_id,node_type_mask,label,is_target
0,0,0,link,1.0,0
1,1,0,link,0.0,0
2,2,0,link,1.0,0
3,3,0,link,1.0,0
4,4,0,link,1.0,0
...,...,...,...,...,...
1586,16,40,entity_0,NaN,0
1587,17,40,entity_2,NaN,0
1588,18,40,link,0.0,0
1589,19,40,node,NaN,0


In [13]:
NodeLabel[NodeLabel.community_id == 1]

,node_id,community_id,node_type_mask,label,is_target
59,0,1,link,1.0,0
60,1,1,link,1.0,0
61,2,1,link,0.0,0
62,3,1,link,1.0,0
63,4,1,link,1.0,0
64,5,1,link,1.0,0
65,6,1,entity_2,NaN,0
66,7,1,entity_2,NaN,0
67,8,1,entity_0,NaN,0
68,9,1,link,0.0,0


In [14]:
DataEdgeWeight

,src,dst,id,edge_weight,edge_importance
0,2,42,0,0.104021,2
1,39,42,0,0.106907,2
2,34,42,0,0.098370,2
3,36,42,0,0.119595,2
4,9,42,0,0.122247,2
...,...,...,...,...,...
6683,17,11,40,0.132183,1
6684,17,12,40,0.091196,1
6685,17,3,40,0.089379,1
6686,2,15,40,0.000000,0


In [15]:
DataSeed

,seed,id,y,yhat
0,19,0,1.0,0.352880
1,39,1,0.0,0.971888
2,18,2,1.0,0.936662
3,36,3,1.0,0.992670
4,57,4,1.0,0.017502
5,36,5,1.0,0.987408
6,31,6,1.0,0.004571
7,35,7,0.0,0.001473
8,43,8,0.0,0.002971
9,10,9,0.0,0.038161


In [16]:
mapping = {'link': 'txn', 'node': 'buyer', 'entity_0': 'pmt_token', 'entity_1': 'email', 'entity_2': 'ship_addr'}

In [17]:
NodeLabel

,node_id,community_id,node_type_mask,label,is_target
0,0,0,link,1.0,0
1,1,0,link,0.0,0
2,2,0,link,1.0,0
3,3,0,link,1.0,0
4,4,0,link,1.0,0
...,...,...,...,...,...
1586,16,40,entity_0,NaN,0
1587,17,40,entity_2,NaN,0
1588,18,40,link,0.0,0
1589,19,40,node,NaN,0


In [18]:
DataEdgeWeight['edge_weight'] = edge_btw_df['hybrid']

In [20]:
from collections import Counter
Counter(edge_btw_df.importance)

Counter({0.0: 854,
         1.0: 342,
         2.0: 643,
         1.75: 167,
         0.75: 207,
         0.25: 46,
         1.6666666669999999: 102,
         0.833333333: 96,
         0.333333333: 45,
         1.5: 111,
         1.333333333: 89,
         1.166666667: 14,
         0.666666667: 42,
         0.875: 22,
         0.375: 78,
         1.25: 74,
         0.5: 228,
         1.8333333330000001: 27,
         0.166666667: 27,
         1.625: 22,
         1.375: 5,
         0.625: 24,
         1.125: 34,
         1.875: 32,
         0.125: 13})

In [23]:
# hybrid weights, (pink_skyblue)
for community_id in DataSeed.id:
# for community_id in [1,4,13,22,31,25]:
#for community_id in [11]:
#for community_id in [22]:
    legend_or_not = True

    print('community_id:', community_id)
    
    plt.rcParams["figure.figsize"] = (32,32)
    
    G=nx.Graph()

    edge_btw_df_sub = edge_btw_df[edge_btw_df.id == community_id]
    
    G = nx.from_pandas_edgelist(edge_btw_df_sub, source = 'src', target = 'dst')
    #,create_using=nx.MultiDiGraph(), edge_attr = True)
   
    NodeLabel_sub = NodeLabel[NodeLabel.community_id == community_id]
    mapping = {'link': 'txn', 'node': 'buyer', 'entity_0': 'pmt_token', 'entity_1': 'email', 'entity_2': 'ship_addr'}
    NodeLabel_sub = NodeLabel_sub.replace(mapping)
    
    grey_buyer = NodeLabel_sub[NodeLabel_sub.node_type_mask == 'buyer'].node_id.to_list()
    grey_pmt_token = NodeLabel_sub[NodeLabel_sub.node_type_mask == 'pmt_token'].node_id.to_list()
    grey_email = NodeLabel_sub[NodeLabel_sub.node_type_mask == 'email'].node_id.to_list()
    grey_ship_addr = NodeLabel_sub[NodeLabel_sub.node_type_mask == 'ship_addr'].node_id.to_list()
    green_txn_legit = NodeLabel_sub[(NodeLabel_sub.node_type_mask == 'txn')&(NodeLabel_sub.label == 0.0)].node_id.to_list()
    orange_txn_fraud = NodeLabel_sub[(NodeLabel_sub.node_type_mask == 'txn')&(NodeLabel_sub.label == 1.0)].node_id.to_list()
    red_cross_node_2_explain = DataSeed[DataSeed.id == community_id].seed.to_list()
    if red_cross_node_2_explain[0] in green_txn_legit:
        green_txn_legit.remove(red_cross_node_2_explain[0])
    else:
        orange_txn_fraud.remove(red_cross_node_2_explain[0])
    
    #print('different lists:', grey_buyer, grey_pmt_token, grey_email, grey_ship_addr, green_txn_legit, orange_txn_fraud, red_cross_node_2_explain)
    
    
    edge_btw_df_sub['edge'] = list(zip(edge_btw_df_sub.src, edge_btw_df_sub.dst))
    edge_weight_dict = dict(zip(edge_btw_df_sub.edge, edge_btw_df_sub.hybrid))
    
    edge_weights = []
    for e in G.edges:
        try:    
            edge_weights.append(edge_weight_dict[e])
        except:
            edge_weights.append(edge_weight_dict[(e[1], e[0])])
            
    
    #print(edge_weight_dict)
    all_edge_weight_dict = edge_weight_dict.copy()
    for e, w in edge_weight_dict.items():
        all_edge_weight_dict[e[1], e[0]] = w
    
    #print(all_edge_weight_dict)
    
    labels = dict(zip(NodeLabel_sub.node_id, NodeLabel_sub.node_id))
    pos = nx.spring_layout(G)
    ax=plt.gca()
    
    nx.draw_networkx_nodes(G, pos=pos, nodelist=grey_buyer, node_color='lightgrey', 
                           node_shape='s', label = 'buyer', node_size = 8000, alpha = 0.8)
    nx.draw_networkx_nodes(G, pos=pos, nodelist=grey_pmt_token, node_color='lightgrey', node_shape='d', 
                           label = 'pmt_token', node_size = 8000, alpha = 0.8)
    nx.draw_networkx_nodes(G, pos=pos, nodelist=grey_email, node_color='lightgrey', node_shape='>', 
                           label = 'email', node_size = 8000, alpha = 0.8)
    nx.draw_networkx_nodes(G, pos=pos, nodelist=grey_ship_addr, node_color='lightgrey', node_shape='^', 
                           label = 'ship_addr', node_size = 15000)#, alpha = 0.8)
    nx.draw_networkx_nodes(G, pos=pos, nodelist=green_txn_legit, node_color='skyblue',  ##
                           node_shape='o', label = 'txn_legit', node_size = 8000, alpha = 0.8)
    nx.draw_networkx_nodes(G, pos=pos, nodelist=orange_txn_fraud, node_color='pink',  ##
                           node_shape='o', label = 'txn_fraud', node_size = 8000, alpha = 0.8)
    nx.draw_networkx_nodes(G, pos=pos, nodelist=red_cross_node_2_explain, node_color='red', node_shape='x', 
                           label = 'node_to_explain', linewidths = 28, node_size = 8000, alpha = 0.8)
    
    nx.draw_networkx_edges(G, pos=pos, width = [_*300 for _ in list(edge_weights)], 
                           node_size = 3000, edge_color = 'grey', connectionstyle="arc3,rad=2", min_source_margin = 20,
                          min_target_margin=20)

    nx.draw_networkx_labels(G, pos, labels, font_size=60, font_color="black")
    # Setting it to how it was looking before.                                                                                                              
    plt.axis('off')

    if legend_or_not == True:
        #plt.legend(loc='lower left', fontsize = 'xx-small')
        plt.legend(scatterpoints = 1, loc = 'best', fontsize = 'xx-small', prop={'size':70})

    plt.subplots_adjust(left=0.00001, right=0.999999, top=0.999999, bottom=0.00001)



    plt.savefig('./results/community{}_structure_hybrid_{}_pink_skyblue.pdf'.format(community_id, legend_or_not))
    plt.clf()
    plt.cla()
    plt.close()



community_id: 0
community_id: 1
community_id: 2
community_id: 3
community_id: 4
community_id: 5
community_id: 6
community_id: 7
community_id: 8
community_id: 9
community_id: 10
community_id: 11
community_id: 12
community_id: 13
community_id: 14
community_id: 15
community_id: 16
community_id: 17
community_id: 18
community_id: 19
community_id: 20
community_id: 21
community_id: 22
community_id: 23
community_id: 24
community_id: 25
community_id: 26
community_id: 27
community_id: 28
community_id: 29
community_id: 30
community_id: 31
community_id: 32
community_id: 33
community_id: 34
community_id: 35
community_id: 36
community_id: 37
community_id: 38
community_id: 39
community_id: 40


In [22]:
# hybrid weights
# This one has special topology, needs additional treatment to drop unwanted nodes in the vis.
#for community_id in DataSeed.id:
legend_or_not = False

nodes_to_del = [50, 37, 19, 7, 14, 22, 52, 35, 16, 8]
for community_id in [4]:

    print('community_id:', community_id)
    
    plt.rcParams["figure.figsize"] = (32,32)
    
    G=nx.Graph()

    edge_btw_df_sub = edge_btw_df[edge_btw_df.id == community_id]
    
    G = nx.from_pandas_edgelist(edge_btw_df_sub, source = 'src', target = 'dst')
    #,create_using=nx.MultiDiGraph(), edge_attr = True)
   
    NodeLabel_sub = NodeLabel[NodeLabel.community_id == community_id]
    mapping = {'link': 'txn', 'node': 'buyer', 'entity_0': 'pmt_token', 'entity_1': 'email', 'entity_2': 'ship_addr'}
    NodeLabel_sub = NodeLabel_sub.replace(mapping)
    
    grey_buyer = NodeLabel_sub[NodeLabel_sub.node_type_mask == 'buyer'].node_id.to_list()
    grey_buyer = [_ for _ in grey_buyer if _ not in nodes_to_del]
    
    grey_pmt_token = NodeLabel_sub[NodeLabel_sub.node_type_mask == 'pmt_token'].node_id.to_list()
    grey_pmt_token = [_ for _ in grey_pmt_token if _ not in nodes_to_del]
    
    grey_email = NodeLabel_sub[NodeLabel_sub.node_type_mask == 'email'].node_id.to_list()
    grey_email = [_ for _ in grey_email if _ not in nodes_to_del]
    
    grey_ship_addr = NodeLabel_sub[NodeLabel_sub.node_type_mask == 'ship_addr'].node_id.to_list()
    grey_ship_addr = [_ for _ in grey_ship_addr if _ not in nodes_to_del]
    
    green_txn_legit = NodeLabel_sub[(NodeLabel_sub.node_type_mask == 'txn')&(NodeLabel_sub.label == 0.0)].node_id.to_list()
    green_txn_legit = [_ for _ in green_txn_legit if _ not in nodes_to_del]
    
    orange_txn_fraud = NodeLabel_sub[(NodeLabel_sub.node_type_mask == 'txn')&(NodeLabel_sub.label == 1.0)].node_id.to_list()
    orange_txn_fraud = [_ for _ in orange_txn_fraud if _ not in nodes_to_del]
    
    red_cross_node_2_explain = DataSeed[DataSeed.id == community_id].seed.to_list()
    grey_pmt_token = [_ for _ in grey_pmt_token if _ not in nodes_to_del]
    
    if red_cross_node_2_explain[0] in green_txn_legit:
        green_txn_legit.remove(red_cross_node_2_explain[0])
    else:
        orange_txn_fraud.remove(red_cross_node_2_explain[0])
    
    #print('different lists:', grey_buyer, grey_pmt_token, grey_email, grey_ship_addr, green_txn_legit, orange_txn_fraud, red_cross_node_2_explain)
    
    
    edge_btw_df_sub['edge'] = list(zip(edge_btw_df_sub.src, edge_btw_df_sub.dst))
    edge_weight_dict = dict(zip(edge_btw_df_sub.edge, edge_btw_df_sub.hybrid))
    
    edge_weights = []
    for e in G.edges:
        try:    
            edge_weights.append(edge_weight_dict[e])
        except:
            edge_weights.append(edge_weight_dict[(e[1], e[0])])
            
    
    #print(edge_weight_dict)
    all_edge_weight_dict = edge_weight_dict.copy()
    for e, w in edge_weight_dict.items():
        all_edge_weight_dict[e[1], e[0]] = w
    
    #print(all_edge_weight_dict)
    
    labels = dict(zip(NodeLabel_sub.node_id, NodeLabel_sub.node_id))
    pos = nx.spring_layout(G)
    ax=plt.gca()
    
    nx.draw_networkx_nodes(G, pos=pos, nodelist=grey_buyer, node_color='lightgrey', 
                           node_shape='s', label = 'buyer', node_size = 8000, alpha = 0.8)
    nx.draw_networkx_nodes(G, pos=pos, nodelist=grey_pmt_token, node_color='lightgrey', node_shape='d', 
                           label = 'pmt_token', node_size = 8000, alpha = 0.8)
    nx.draw_networkx_nodes(G, pos=pos, nodelist=grey_email, node_color='lightgrey', node_shape='>', 
                           label = 'email', node_size = 8000, alpha = 0.8)
    nx.draw_networkx_nodes(G, pos=pos, nodelist=grey_ship_addr, node_color='lightgrey', node_shape='^', 
                           label = 'ship_addr', node_size = 15000)#, alpha = 0.8)
    nx.draw_networkx_nodes(G, pos=pos, nodelist=green_txn_legit, node_color='skyblue', 
                           node_shape='o', label = 'txn_legit', node_size = 8000, alpha = 0.8)
    nx.draw_networkx_nodes(G, pos=pos, nodelist=orange_txn_fraud, node_color='pink', 
                           node_shape='o', label = 'txn_fraud', node_size = 8000, alpha = 0.8)
    nx.draw_networkx_nodes(G, pos=pos, nodelist=red_cross_node_2_explain, node_color='red', node_shape='x', 
                           label = 'node_to_explain', linewidths = 28, node_size = 8000, alpha = 0.8)
    
    nx.draw_networkx_edges(G, pos=pos, width = [_*300 for _ in list(edge_weights)], 
                           node_size = 3000, edge_color = 'grey', connectionstyle="arc3,rad=2", min_source_margin = 20,
                          min_target_margin=20)

    nx.draw_networkx_labels(G, pos, labels, font_size=60, font_color="black")
    # Setting it to how it was looking before.                                                                                                              
    plt.axis('off')

    if legend_or_not == True:
        #plt.legend(loc='lower left', fontsize = 'xx-small')
        plt.legend(scatterpoints = 1, loc = 'best', fontsize = 'xx-small', prop={'size':70})

    plt.subplots_adjust(left=0.00001, right=0.999999, top=0.999999, bottom=0.00001)



    plt.savefig('./results/community{}_structure_hybrid_{}_skyblue_pink_special.pdf'.format(community_id, legend_or_not))
    plt.clf()
    plt.cla()
    plt.close()



community_id: 4
